## Agent Framework Context Providers


### Context Providers 

**Context Providers** are the mechanisms you can implement to enable memory and persistent context for the agents.  They act as middleware that runs before and after every agent invocation, allowing you to inject relevant context into the conversation and extract information for future use.

Agent Framework provides built-in conversation history through threads, and you can serialize threads to persist them across sessions. However, these are fundamentally constrained by the LLM's context window. 
Context providers solve this by moving memory and knowledge storage outside the token-limited conversation into external systems designed specifically for information retrieval and management.

In the following tutorial, you will learn how to use **Azure AI Search** with both custom search tools and the `AzureAISearchContextProvider` (automatic context injection). Azure AI Search is excellent for grounding agents in enterprise knowledge and relevant documentation retrieval. It solves the critical challenge of ensuring agents provide accurate, source-backed answers rather than generating potentially incorrect information from the LLM's training data alone.

Check out the [AF Context Provider Samples](https://github.com/microsoft/agent-framework/tree/main/python/samples/getting_started/context_providers) to learn about the implementation basics of other context providers, such as Mem0 for long-term memory and personalization, and Redis for high-performance state management and multi-agent coordination.

### Example 1: Single Agent with a Custom Search Tool
In this section, you will learn how to build a retrieval-augmented generation (RAG) agent with Microsoft Agent Framework. 

First, go to the notebook `03.0-setup-guide.ipynb` for a step-by-step guide on how to deploy all the necessary resources to create the Azure AI Search Index.


Now that we've vectorized our documents and created an index, we can build a Search Agent with Microsoft Agent Framework. This agent will leverage the Azure AI Search index to retrieve relevant information from your documents in response to user queries. In this section, you'll:

- Connect your code to the Azure AI Search index you created earlier
- Define and configure a search agent with the appropriate tools and resources
- Interact with the agent to perform intelligent, context-aware document retrieval

By the end of this lab, you’ll understand the Retrieval-Augmented Generation (RAG) pattern and how to implement it.

For the following exercise, make sure to add the following environment variables to your .env file:

```python
AZURE_SEARCH_ENDPOINT=https://<search-service-name>.search.windows.net
INDEX_NAME=<index-name>-index
SEARCH_API_KEY=<search-service-key>
```

In [ ]:
import os
import json
from dotenv import load_dotenv
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential
from azure.core.exceptions import HttpResponseError
from azure.identity.aio import DefaultAzureCredential, get_bearer_token_provider

load_dotenv()

project_endpoint = os.getenv("AZURE_AI_PROJECT_ENDPOINT")
deployment_name = os.getenv("AZURE_AI_MODEL_DEPLOYMENT_NAME")
api_version = os.getenv("AZURE_AI_API_VERSION")
credential = DefaultAzureCredential()

# Get token with the correct audience for Azure OpenAI
token_provider = get_bearer_token_provider(
    credential,
    "https://cognitiveservices.azure.com/.default"  # Correct audience for Azure OpenAI
)

search_key = os.getenv("SEARCH_API_KEY")
search_endpoint = os.getenv("AZURE_SEARCH_ENDPOINT")
index1_name = os.getenv("INDEX_NAME")


You’re creating two clients:

- `AzureOpenAIChatClient` connects to your chat model deployment using the Foundry project endpoint, Azure credential, API version, and deployment name.
- `SearchClient` connects to your Azure AI Search index using its endpoint, index name and key, allowing you to retrieve relevant documents.

Together these create a RAG workflow: fetch context from search, then pass it to the model for grounded answers.

In [ ]:
from agent_framework.azure import AzureAIAgentClient
from azure.identity.aio import AzureCliCredential
from agent_framework.azure import AzureOpenAIChatClient


client = AzureOpenAIChatClient(
    api_version=api_version,
    azure_endpoint=project_endpoint,
    azure_ad_token_provider=token_provider,
)

# Initialize search client with correct credentials for the first index
search_client = SearchClient(
    endpoint=search_endpoint,
    index_name=index1_name,
    credential=AzureKeyCredential(search_key)
)

The `AzureAISearchTool` wrapper that turns your SearchClient into a callable “tool” for an agent. It’s initialized with an existing Azure AI Search SearchClient. 

The `search(query, top)` method runs a semantic search against your index, collects each hit’s chunk content and `@search.score`, finally returning them as a JSON-formatted string for easy injection into a prompt. If Azure Search returns an error (HttpResponseError), it prints basic diagnostics and returns a JSON error payload instead:

In [ ]:
class AzureAISearchTool:
    """Function tool to search Azure AI Search index"""
   
    def __init__(self, search_client: SearchClient):
        self.search_client = search_client
   
    def search(self, query: str, top: int = 5) -> str:
        """
        Search the Contoso HR knowledge base for detailed information about health plans, policies, and benefits.
        
        This tool provides access to in-depth information about:
        - Northwind Health Plus and Northwind Health Standard plan details
        - Coverage rules, exclusions, and limitations
        - Costs, deductibles, and out-of-pocket maximums
        - Eligibility requirements and enrollment information
        - Company-specific HR policies and benefits
        """
        try:
            results = self.search_client.search(
                query,
                top=top,
                query_type="semantic"
            )
           
            documents = []
            for doc in results:
                documents.append({
                    "chunk": doc.get('chunk', ''),
                    "score": doc.get('@search.score', 0)
                })
           
            # Return formatted results
            return json.dumps(documents, indent=2)
           
        except HttpResponseError as e:
            print("HTTP status:", e.status_code if hasattr(e, "status_code") else "N/A")
            if e.response is not None:
                try:
                    print("Response text:", e.response.text())
                except Exception:
                    print("Could not get response.text()")
            return json.dumps({"error": str(e)})

Finally, we create a simple agent and run it with a user question. The agent uses the chat model for reasoning and the Azure AI Search tool for context retrieval. It follows the given instructions to stay factual and cite sources. 

When you run the cell, you should see a grounded response based on the indexed documents:

In [ ]:
# Initialize the search tool for the agent providing the search client
tools = AzureAISearchTool(search_client=search_client)

agent = client.create_agent(
    name="employee_agent",
    instructions="""
    You are an HR Assistant for Contoso Electronics specializing in employee health plans. Your role is to answer in-depth and technical questions about the two available health plans:
    - Northwind Health Plus Plan
    - Northwind Health Standard Plan

    - You must invoke the custom search tool before responding
    - Never rely on prior knowledge or assumptions
    - Always retrieve information from the search tool first, even for simple questions
    - If the search returns no results or insufficient information, explicitly acknowledge this

    - Base responses exclusively on information retrieved from the search tool
    - Do not infer or fabricate plan details
    - Be concise but thorough: explain differences, coverage details, eligibility, costs, and tradeoffs
    """,
    
    tools=[tools.search]
)

# Minimal run sample with a single question
user_question = "What are out-of-network copays?"
result = await agent.run(
    user_question,
    )

print(result)

### Exercise 1 - Single agent with multiple context providers

We are working with two distinct knowledge bases, each serving a different purpose for our HR Assistant agent:

**Index 1: Health Plans Documentation (Specialized)** - detailed formal documentation about two specific health plans - Northwind Health Plus Plan and Northwind Health Standard Plan

This index includes detailed information on plans' formalities, specific coverage details and exclusions, eligibility requirements and enrollment procedures, etc.

*Purpose*: Specialized, technical resource for answering detailed questions about health plan specifics and comparisons.

**Index 2: General HR Knowledge (Context Provider)** - broad HR information about Contoso Electronics - employee benefits and plan packages overview, policies and employee handbook guidelines, wellness reimbursement programs and role descriptions.

*Purpose*: Provides general HR context and handles common employee inquiries about company policies and benefits.

#### Task

Our goal is to create an HR Assistant agent that:
1. Is grounded in the general HR context from Index 2 (context provider)
2. Automatically invokes Index 1 as a custom search tool when users ask technical, specialized questions about health plan comparisons or specific plan details
3. Combines both knowledge sources to provide comprehensive and accurate answers

This approach ensures:
1. **Efficiency and Cost Optimization** - the specialized health plans index is significantly larger and querying it on every request would be computationally expensive, most user queries don't require this level of detail. Using it as a custom tool only when needed optimizes performance and reduces API costs.
2. **Scalability and Maintainability** -  allows additional specialized tools to be added later without bloating the base context. Health plan documents can be updated independently, and the general HR context can be reused for different specialized agents.

Let's create a second index following the same process as previously in Azure portal. This will give your agent two separate knowledge sources to search from.

#### Steps

1. **Upload documents** from the `data/index2` folder to a new container in your storage account
   - Create a new container
   - Upload the files

2. **Create the second index** in Azure AI Search
   - Follow the exact same steps you used to create the first index
   - Use the new container as your data source
   - Configure vectorization with your **text-embedding-3-large** deployment
   - Give it a distinct name

*Result*: You'll now have two search indexes that your agent can use as separate context providers. Note down **the name of the second index** - this is the only value you'll need to connect it to your agent.


#### Azure AI Search Context Provider

The `AzureAISearchContextProvider` is a production-ready context provider that automatically retrieves relevant documents from Azure AI Search and injects them as context into your agent before every invocation. It abstracts away the complexity of manually managing search operations, retrieval logic, and context injection:


```python
from agent_framework.azure import AzureAISearchContextProvider

search_provider = AzureAISearchContextProvider(
    endpoint=search_endpoint,
    index_name=index_name,
    api_key=api_key,  # Use api_key for API key auth, or credential for managed identity
    credential=AzureKeyCredential(search_key),
    mode="semantic",  # Default mode
    top_k=3,  # Retrieve top 3 most relevant documents
)

agent = client.create_agent(
    instructions="""
    You are a helpful assistant tasked with answering the user’s question accurately and concisely. Always consult the available search tool to retrieve relevant information before responding.
    Base your answer only on the retrieved content and do not make assumptions.
    If the information is missing or unclear, say so explicitly.
    """,
    
    context_providers=[search_provider]     # Grounding provided as a tool
)

# Minimal run sample with a single question
user_question = "Will the companies that were already certified have to recertify?"
result = await agent.run(
    user_question,
    )

print(result)
```

`AzureAISearchContextProvider` is recommended for production RAG applications where agents must consistently provide accurate, source-backed answers from indexed documentation. In the next steps, we wi

### Example 2: Single Agent with multiple knowledge sources
In the following example, we ground the agent in the basic HR knowledge via the context provider and then use the custom search tool we built previously to be invoked whenever the agent decides it is relevant.

#### Sample overview:

1. **`ChatAgent`**

This implementation demonstrates the recommended pattern for building production-grade AI agents using AF's `ChatAgent` class. While simpler patterns like `client.create_agent()` exist, `ChatAgent` is the proper foundation for applications requiring stateful orchestration, tool calling capabilities, context provider integration, and multi-turn conversation management.

2. **`async`**

We use `async with` statement to manage two resources: the `search_provider` (which maintains connections to Azure AI Search) and the `agent` itself (which manages its internal resources and lifecycle). This pattern ensures both are properly initialized and cleaned up, so when you exit, it automatically cleans up those resources (closing connections, releasing handles). This is critical for async operations because it guarantees cleanup happens even if errors occur, preventing resource leaks in long-running applications.

3. **`run_stream`**

Streaming returns an async iterator that yields response chunks as the LLM generates them, rather than waiting for the complete response. This ensures that large responses are processed incrementally without buffering everything in memory. Each chunk is an `AgentRunResponseUpdate` object containing incremental text that we print immediately with `flush=True` to show real-time progress.

In [ ]:
import os
from agent_framework import ChatAgent
from agent_framework.azure import AzureAISearchContextProvider
from agent_framework import function_middleware

# Make sure you've added the newly created index name to the .env file and reloaded the window 
index2 = os.getenv("INDEX2_NAME")

# simple middleware to check when the search tool gets invoked
@function_middleware 
async def simple_tool_mw(context, next): 
    """Function middleware that runs before and after function calls."""
    print(f"[Function Middleware] BEFORE CALLING: {context.function.name}")  
    await next(context)
    print(f"[Function Middleware] After calling: {context.function.name}") 


USER_INPUTS = [
    "What's ERISA?",
    "What does Northwind Standard not cover?",
    "Can independent contractor services be covered?"
]

search_provider = AzureAISearchContextProvider(
    endpoint=search_endpoint,
    index_name=index2,
    api_key=credential,
    credential=AzureKeyCredential(search_key),
    mode="semantic",
    top_k=3,
)

async with (
    search_provider,
    ChatAgent(
        chat_client=client,    # instantiate the `ChatAgent` using the `AzureOpenAIChatClient` created earlier
        name="HRAgent",
        instructions=(
            """
            You are a helpful HR assistant for Contoso Electronics employees.

            ## Available Information Sources:

            1. **Background Knowledge** (automatically available): General employee benefits, company policies, wellness programs, role descriptions
            2. **Health Plan Search Tool** (you must explicitly call this): Technical details about Northwind Health Plus and Northwind Health Standard plans

            Call the search tool when:
            - Questions are about Northwind Health Plus or Standard plans
            - You need specific details about coverage, costs, or benefits
            - Your background knowledge doesn't have enough information to answer accurately
            - You're uncertain about answer

            ## How to Respond:
            
            1. If the question is about health plan details → Call the search tool FIRST
            2. Use the search results to answer the question
            3. Answer ONLY using facts from search results or background knowledge
            4. Keep responses to 2-3 sentences maximum. Be concise and direct.
            5. If information isn't available, say: "I don't have that information. Please contact HR."

            Keep answers brief but complete.
            """
            ),
        context_providers=[search_provider],
        tools=[tools.search],
        middleware=[simple_tool_mw]
    ) as hr_agent,
):

    for user_input in USER_INPUTS:
        print(f"User: {user_input}")
        print("Agent: ", end="", flush=True)

        # Stream response
        async for chunk in hr_agent.run_stream(user_input):
            if chunk.text:
                print(chunk.text, end="", flush=True)

        print("\n")